# Lab 07

## Zadanie 1

Przygotuj demo programu znajdującego miejsca zerowe metodą Newtona. Wykorzystując `argparse` ([link](https://docs.python.org/3.6/library/argparse.html#module-argparse)) lub `optparse` ([link](https://docs.python.org/3.6/library/optparse.html)) obsłuż:
- ustalane punktu startowego,
- wielkość kroku w pochodnej,
- ilość kroków metody,
- dokładność
- pomoc

Program uruchamiamy podając, np.:

`./newton.py x**2+x+1 -h 0.00001`

In [ ]:
import argparse
import sympy
from sympy.parsing.sympy_parser import standard_transformations,implicit_multiplication_application
from sympy.parsing.sympy_parser import parse_expr


parser=argparse.ArgumentParser(description="Newton Method")
parser.add_argument('polynomial',action='store',type=str,help='Polynomial without space variable as x')
parser.add_argument('-x0','--xstart',dest='x0',action='store',type=float,default=0.0,help='Starting point')
parser.add_argument('-m','--steps',dest='steps',action='store',type=int,default=100,help='Number steps of Newton Method')
parser.add_argument('-e','--epsilon',dest='epsilon',action='store',type=float,default=10e-6,help='Acceptable error to 0')

args=parser.parse_args()

transformations = (standard_transformations +
    (implicit_multiplication_application,))

parsed=parse_expr(args.polynomial, transformations=transformations)
steps=args.steps
epsilon=args.epsilon
x_prev=args.x0

def newtons_method(x_prev,f,epsilon,max_iterations):
    derivative = f.diff('x')
    for i in range(max_iterations):
        y = f.subs('x',x_prev)
        der_at_point=derivative.subs('x', x_prev)
        if abs(der_at_point) < 1e-10:
            print("Derivative too close to zero.")
            return None
        x1 = x_prev - y / der_at_point      
        if abs(x1 - x_prev) <= epsilon:
            return x_prev
        x_prev = x1
     
    return x_prev


print(newtons_method(x_prev,parsed,epsilon,steps))

##PLIK zad1.py w lab07 files

In [46]:
!py lab07files/zad1.py x**2-4x+4 -x0 -1

1.99998855590820


## Zadanie 2

Zadanie należy wykonać wykorzystując program `BagOfWords` z poprzednich zajęć.


Przerób ją tak, żeby interpunkcja, cyfry i wszelkie inne znaki nie przeszkadzały w parsowaniu tekstu. Uruchom ja na [tekście hamleta](http://www.gutenberg.org/cache/epub/1787/pg1787.txt). Ile razy występuje słowo hamlet? Jak brzmi 10 najczęściej występujących słów?

In [11]:
import io
import re
from collections import OrderedDict
class BagOfWords(object):
    
    def __init__(self, obj):
        _text=""
        self._dict={}
        if isinstance(obj,str):
            _text=obj
        elif isinstance(obj, io.TextIOBase):
            _text=obj.read()
        _text=_text.lower()
        _text=re.sub(r'[^A-Za-z \n]','',_text)
        print(_text)
        for w in _text.split():
                self._dict[w]=self._dict.get(w,0)+1
        
    def __str__(self):
        self.order_dict()
        return ", ".join([f'{k}:{v}' for k,v in self._dict.items()])
    
    def __getitem__(self, index):
        if isinstance(index, str):
            return self._dict.get(index,0)
        else:
            raise ValueError

    def __setitem__(self, index, newvalue):
        if isinstance(index,str) and isinstance(newvalue,int):
            self._dict[index]=newvalue
        else:
            raise ValueError
    
    def __iter__(self):
        self.order_dict()
        for i in self._dict:
            yield i

    def __add__(self,other):
        if isinstance(other,BagOfWords):
            dictionary_temp=self._dict.copy()
            for k,v in other._dict.items():
                dictionary_temp[k]=dictionary_temp.get(k,0)+v       
            new_temp=BagOfWords('')
            new_temp._dict=dictionary_temp
            return new_temp
        else:
            raise NotImplemented
    def order_dict(self):
        self._dict=dict(sorted(self._dict.items(),key=lambda x:x[1],reverse=True))

    def top10(self):
        self.order_dict()
        return ", ".join([f'{k}:{v}' for k,v in list(self._dict.items())[:10]])

g=BagOfWords(open("lab07files/hamlet.txt",encoding="utf8"))
print(g.top10())
g['hamlet']


the project gutenberg ebook of hamlet
    
this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrictions
whatsoever you may copy it give it away or reuse it under the terms
of the project gutenberg license included with this ebook or online
at wwwgutenbergorg if you are not located in the united states
you will have to check the laws of the country where you are located
before using this ebook

title hamlet


author william shakespeare

release date june   ebook 
                most recently updated may  

language english



 start of the project gutenberg ebook hamlet 
this etext is a typocorrected version of shakespeares hamlet
project gutenberg file wstxt



this ebook was one of project gutenbergs early files produced at a
time when proofing methods and tools were not well developed there
is an improved edition of this title which may be viewed as ebook
 at httpswwwgutenbergorgebooks



this etext fi

115

## Zadanie 3

Wykorzystując `pickle` zapisz i odczytaj klasy z poprzedniego zadania nakarmonej Hamletem. Porównaj metody i rozmiar.

In [56]:
import pickle
with open('lab07files/object_serialized.pkl', 'wb') as file:
    pickle.dump(g, file)

In [57]:
with open('lab07files/object_serialized.pkl','rb') as file:
    data=pickle.load(file)


In [58]:
import sys
print(sys.getsizeof(g)==sys.getsizeof(data))
dir(g)==dir(data)

True


True

## Zadanie 4

Wykorzystując [https://gist.github.com/pamelafox/986163](https://gist.github.com/pamelafox/986163) podaj aktualną godzinę we:
- wszystkich krajach, wyświetlając je zgrupowane względem kontynentów,
- przeprowadź symulacyjne wyświetlanie kolejnych krajów w miarę jak w danym kraju wybija północ, opóżnienie wyświetlania ustaw proporcjonalne do realnego czasu

In [51]:
import requests
gist_url = 'https://gist.githubusercontent.com/pamelafox/986163/raw/eeeabff229831334f8a84bdddd5768cc84c50bd4/countryinfo.py'
response = requests.get(gist_url)

if response.status_code == 200:
    gist_content = response.text
    print(gist_content)
countries = eval(gist_content[12:])


countries = [
{'timezones': ['Europe/Andorra'], 'code': 'AD', 'continent': 'Europe', 'name': 'Andorra', 'capital': 'Andorra la Vella'},
{'timezones': ['Asia/Kabul'], 'code': 'AF', 'continent': 'Asia', 'name': 'Afghanistan', 'capital': 'Kabul'},
{'timezones': ['America/Antigua'], 'code': 'AG', 'continent': 'North America', 'name': 'Antigua and Barbuda', 'capital': "St. John's"},
{'timezones': ['Europe/Tirane'], 'code': 'AL', 'continent': 'Europe', 'name': 'Albania', 'capital': 'Tirana'},
{'timezones': ['Asia/Yerevan'], 'code': 'AM', 'continent': 'Asia', 'name': 'Armenia', 'capital': 'Yerevan'},
{'timezones': ['Africa/Luanda'], 'code': 'AO', 'continent': 'Africa', 'name': 'Angola', 'capital': 'Luanda'},
{'timezones': ['America/Argentina/Buenos_Aires', 'America/Argentina/Cordoba', 'America/Argentina/Jujuy', 'America/Argentina/Tucuman', 'America/Argentina/Catamarca', 'America/Argentina/La_Rioja', 'America/Argentina/San_Juan', 'America/Argentina/Mendoza', 'America/Argentina/Rio_Gallegos', '

In [52]:
from datetime import datetime
from zoneinfo import ZoneInfo

for i in sorted(countries,key=lambda d: d['continent']):
    for j in i.get('timezones', []):
        print(datetime.now(tz=ZoneInfo(j)),j,i['continent'])


2023-11-13 21:28:23.308827+01:00 Africa/Luanda Africa
2023-11-13 20:28:23.309829+00:00 Africa/Ouagadougou Africa
2023-11-13 22:28:23.309829+02:00 Africa/Bujumbura Africa
2023-11-13 21:28:23.310827+01:00 Africa/Porto-Novo Africa
2023-11-13 22:28:23.310827+02:00 Africa/Gaborone Africa
2023-11-13 21:28:23.310827+01:00 Africa/Kinshasa Africa
2023-11-13 22:28:23.310827+02:00 Africa/Lubumbashi Africa
2023-11-13 21:28:23.310827+01:00 Africa/Brazzaville Africa
2023-11-13 20:28:23.311828+00:00 Africa/Abidjan Africa
2023-11-13 21:28:23.311828+01:00 Africa/Douala Africa
2023-11-13 19:28:23.311828-01:00 Atlantic/Cape_Verde Africa
2023-11-13 23:28:23.311828+03:00 Africa/Djibouti Africa
2023-11-13 22:28:23.312832+02:00 Africa/Cairo Africa
2023-11-13 23:28:23.312832+03:00 Africa/Asmera Africa
2023-11-13 23:28:23.312832+03:00 Africa/Addis_Ababa Africa
2023-11-13 21:28:23.312832+01:00 Africa/Libreville Africa
2023-11-13 20:28:23.312832+00:00 Africa/Accra Africa
2023-11-13 20:28:23.313830+00:00 Africa/B

In [ ]:
from collections import defaultdict 
import time

d = defaultdict(list)

def converter_offset(number):
    return number if number <= 14 else number - 24

#2023-11-13 20:55:53.061573+01:00
#2023-11-14 00:25:53.062572+04:30

for i in countries:
    for j in i.get('timezones', []):
        data = str(datetime.now(tz=ZoneInfo(j)))
        num=int(data[26:29])
        offset = num+24 if (num==-11 or num==-10) else num
        d[offset].append(j)

while True:
    if datetime.utcnow().minute==0:
        #how many hours to 24:00
        h=converter_offset(24-datetime.utcnow().hour)
        for i in d[h]:
            print(i,"00:00")
            time.sleep(1)                
        time.sleep(61)
        



Africa/Ouagadougou 00:00
Africa/Abidjan 00:00
Africa/Accra 00:00
Africa/Banjul 00:00
Africa/Conakry 00:00
Africa/Bissau 00:00
Europe/Dublin 00:00
Atlantic/Reykjavik 00:00
Africa/Monrovia 00:00
Africa/Bamako 00:00
Africa/Nouakchott 00:00
Europe/Lisbon 00:00
Atlantic/Madeira 00:00
Africa/Freetown 00:00
Africa/Dakar 00:00
Africa/Sao_Tome 00:00
Africa/Lome 00:00
Atlantic/Canary 00:00
Europe/London 00:00
Atlantic/Cape_Verde 00:00
Atlantic/Azores 00:00
America/Noronha 00:00
America/Argentina/Buenos_Aires 00:00
America/Argentina/Cordoba 00:00
America/Argentina/Jujuy 00:00
America/Argentina/Tucuman 00:00
America/Argentina/Catamarca 00:00
America/Argentina/La_Rioja 00:00
America/Argentina/San_Juan 00:00
America/Argentina/Mendoza 00:00
America/Argentina/Rio_Gallegos 00:00
America/Argentina/Ushuaia 00:00
America/Belem 00:00
America/Fortaleza 00:00
America/Recife 00:00
America/Araguaina 00:00
America/Maceio 00:00
America/Bahia 00:00
America/Sao_Paulo 00:00
America/St_Johns 00:00
America/Santiago 0

## Zadanie 5

Dla klasy `BagOfWords` napisz metody `save` oraz `load` wykorzystujące `json`'a do zapisu i odczytu danych.

In [54]:
import io
import re
import json

from collections import OrderedDict
class BagOfWords(object):
    
    def __init__(self, obj):
        _text=""
        self._dict={}
        if isinstance(obj,str):
            _text=obj
        elif isinstance(obj, io.TextIOBase):
            _text=obj.read()
        _text = re.sub(r'[\[\],\n]', ' ', _text)
        _text=re.sub(r'[^A-za-z ]','',_text)

        
        for w in _text.split():
                self._dict[w]=self._dict.get(w,0)+1
        
    def __str__(self):
        self.order_dict()
        return ", ".join([f'{k}:{v}' for k,v in self._dict.items()])
    
    def __getitem__(self, index):
        if isinstance(index, str):
            return self._dict.get(index,0)
        else:
            raise ValueError

    def __setitem__(self, index, newvalue):
        if isinstance(index,str) and isinstance(newvalue,int):
            self._dict[index]=newvalue
        else:
            raise ValueError
    
    def __iter__(self):
        self.order_dict()
        for i in self._dict:
            yield i

    def __add__(self,other):
        if isinstance(other,BagOfWords):
            dictionary_temp=self._dict.copy()
            for k,v in other._dict.items():
                dictionary_temp[k]=dictionary_temp.get(k,0)+v       
            new_temp=BagOfWords('')
            new_temp._dict=dictionary_temp
            return new_temp
        else:
            raise NotImplemented
    def order_dict(self):
        self._dict=dict(sorted(self._dict.items(),key=lambda x:x[1],reverse=True))

    def top10(self):
        self.order_dict()
        return ", ".join([f'{k}:{v}' for k,v in list(self._dict.items())[:10]])

    def save_json(self):
        with open('lab07files/bag.json', 'w', encoding='utf-8') as f:
            json.dump(self._dict, f, ensure_ascii=False, indent=4)
        
    def load_json(self):
        with open('lab07files/bag.json') as json_file:
            data = json.load(json_file)
            self._dict=data


g=BagOfWords(open("lab07files/hamlet.txt",encoding='utf-8'))
g.save_json()
g2=BagOfWords(open("lab07files/hamlet.txt",encoding='utf-8'))
g2.load_json()
print(str(g)==str(g2))


True
